## Traversing of a tree

This is a program that traverses a linked list computing a sequence of Fibonacci numbers at each node. 

Parallelize this program using parallel region, tasks and other directives. Compare your solution’s complexity compared to the approach without tasks.

Defined libraries, variables, structs and functions:

In [ ]:
#pragma cling load("libomp.so")
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>

#define N 5
#define FS 38

In [ ]:
struct node {
    int data;
    int fibdata;
    struct node* next;
};

In [ ]:
int fib(int n) {
    int x, y;
    if (n < 2) {
      return (n);
    } else {
      x = fib(n - 1);
      y = fib(n - 2);
      return (x + y);
    }
}

In [ ]:
void processwork(struct node* p) {
    int n;
    n = p->data;
    p->fibdata = fib(n);
}

In [ ]:
struct node* init_list(struct node* p) {
    int i;
    struct node* head = NULL;
    struct node* temp = NULL;

    head = (struct node*)malloc(sizeof(struct node));
    p = head;
    p->data = FS;
    p->fibdata = 0;
    for (i=0; i< N; i++) {
       temp  = (struct node*)malloc(sizeof(struct node));
       p->next = temp;
       p = temp;
       p->data = FS + i + 1;
       p->fibdata = i+1;
    }
    p->next = NULL;
    return head;
}

Main part of the program:

In [ ]:
double start, end;
struct node *p=NULL;
struct node *temp=NULL;
struct node *head=NULL;

printf("Process linked list\n");
printf("Each linked list node will be processed by function 'processwork()'\n");
printf("Each ll node will compute %d fibonacci numbers beginning with %d\n",N,FS);      

p = init_list(p);
head = p;

start = omp_get_wtime();
{
    while (p != NULL) {
       processwork(p);
       p = p->next;
    }
}

end = omp_get_wtime();
p = head;
while (p != NULL) {
    printf("%d : %d\n",p->data, p->fibdata);
    temp = p->next;
    free (p);
    p = temp;
}
free (p);

printf("Compute Time: %f seconds\n", end - start);

### After successful execution, you can compare with the solution:

In [ ]:
int num_threads = 2;
omp_set_num_threads(num_threads);

double start, end;
struct node *p=NULL;
struct node *temp=NULL;
struct node *head=NULL;

printf("Process linked list\n");
printf("Each linked list node will be processed by function 'processwork()'\n");
printf("Each ll node will compute %d fibonacci numbers beginning with %d\n",N,FS);      

p = init_list(p);
head = p;

start = omp_get_wtime();

#pragma omp parallel 
{
    #pragma omp master
        printf("Threads:      %d\n", omp_get_num_threads());

        #pragma omp single
        {
            p=head;
            while (p) {
                #pragma omp task firstprivate(p) //first private is required
                {
                    processwork(p);
                }
              p = p->next;
           }
        }
}

end = omp_get_wtime();
p = head;
while (p != NULL) {
    printf("%d : %d\n",p->data, p->fibdata);
    temp = p->next;
    free (p);
    p = temp;
}  
free (p);

printf("Compute Time: %f seconds\n", end - start);